In [9]:
import tvm
import tvm.testing
from tvm import te
import numpy as np

# 全局环境定义

tgt_host = "llvm"
# 如果启用了GPU，则将其更改为相应的GPU，例如：cuda、opencl、rocm
tgt = "cuda"

In [10]:
n = te.var("n")
A = te.placeholder((n,), name="A")
B = te.placeholder((n,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")
print(type(C))

<class 'tvm.te.tensor.Tensor'>


In [26]:
s = te.create_schedule(C.op)
# block_c = s.get_block("C")
# i, = s.get_loops(block_c)
bx, tx = s[C].split(C.op.axis[0], factor=64)

In [12]:
# if tgt == "cuda" or tgt == "rocm" or tgt.startswith("opencl"):
    # s[C].bind(bx, te.thread_axis("blockIdx.x"))
    # s[C].bind(tx, te.thread_axis("threadIdx.x"))

In [13]:
fadd = tvm.build(s, [A, B, C], target_host=tgt_host, name="myadd")

In [15]:
# ctx = tvm.context(tgt, 0)

n = 1024
# a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), ctx)
# b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), ctx)
# c = tvm.nd.array(np.zeros(n, dtype=C.dtype), ctx)
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype))
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype))
c = tvm.nd.array(np.zeros(n, dtype=C.dtype))
fadd(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())

In [ ]:

print(fadd.get_source)
